In [1]:
import sys
import dgl
import dgl.function as fn
import os
import multiprocessing as mp
from tqdm import tqdm
import pdb
import numpy as np
import torch
import torch.nn as nn
import logging
from utils.parser import parse_args
from utils.dataloader import Dataloader
from utils.utils import config, construct_negative_graph, choose_model, load_mf_model, NegativeGraph
from utils.tester import Tester
from models.sampler import NegativeSampler
import wandb

In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default = 'TaoBao', type = str,
                    help = 'Dataset to use')
parser.add_argument('--seed', default = 2022, type = int,
                    help = 'seed for experiment')
parser.add_argument('--embed_size', default = 32, type = int,
                    help = 'embedding size for all layer')
parser.add_argument('--lr', default = 0.05, type = float,
                    help = 'learning rate')
parser.add_argument('--weight_decay', default = 8e-8, type = float,
                    help = "weight decay for adam optimizer")
#######################################################################
parser.add_argument('--model', default ='test',type = str,
                    help = 'model selection')#dgrec base moe test
parser.add_argument('--epoch', default = 1000, type = int,
                    help = 'epoch number')
parser.add_argument('--patience', default = 10, type = int,
                    help = 'early_stop validation')
parser.add_argument('--batch_size', default = 2048, type = int,
                    help = 'batch size')
parser.add_argument('--layers', default = 1, type = int,
                    help = 'layer number')
parser.add_argument('--gpu', default = 0, type = int,
                    help = '-1 for cpu, 0 for gpu:0')
parser.add_argument('--k_list', default = [100, 300], type = list,
                    help = 'topk evaluation')
parser.add_argument('--k', default = 20, type = int,
                    help = 'neighbor number in each GNN aggregation')
parser.add_argument('--neg_number', default = 4, type = int,
                    help = 'negative sampler number for each positive pair')
parser.add_argument('--metrics', default = ['recall', 'hit_ratio', 'coverage'])


parser.add_argument('--sigma', default = 1.0, type = float,
                    help = 'sigma for gaussian kernel')
parser.add_argument('--gamma', default = 2.0, type = float,
                    help = 'gamma for gaussian kernel')
################################################################################
parser.add_argument('--category_balance', default = True, type = bool,
                    help = 'whether make loss category balance')
parser.add_argument('--beta_class', default = 0.9, type = float,
                    help = 'class re-balanced loss beta')
parser.add_argument('--context_code_dim', default = 32, type = int,
                    help = 'interest num')
parser.add_argument('--num_context_codes', default = 32, type = int,
                    help = 'interest dim')
parser.add_argument('--n_experts', default = 5, type = int,
                    help = 'n_experts')
##########################################################################################
parser.add_argument('--wandb_enable', default = True, type = bool,
                    help = 'layer number')
parser.add_argument('--hidden_size', default = 32, type = int,
                        help = 'n_experts')
parser.add_argument('--k_experts', default = 2, type = int,
                            help = 'n_experts')
parser.add_argument('--moe', default = False, type = bool,
                            help = 'layer number')
################################################################################
parser.add_argument('--sub', default = 'rand', type = str,
                            help = 'layer number')

args = parser.parse_args([])

In [3]:
data = args.dataset
if args.gpu >= 0 and torch.cuda.is_available():
        device = 'cuda:{}'.format(args.gpu)
else:
        device = 'cpu'
device = torch.device(device)
args.device = device
dataloader = Dataloader(args, data, device)
graph = dataloader.train_graph

hid_dim = args.embed_size
user_embedding = torch.nn.Parameter(torch.randn(graph.nodes('user').shape[0], hid_dim))
item_embedding = torch.nn.Parameter(torch.randn(graph.nodes('item').shape[0], hid_dim))
node_features = {'user': user_embedding, 'item': item_embedding}
etype=('item', 'rated by', 'user')




100%|██████████| 136710/136710 [00:00<00:00, 4557076.78it/s]


In [97]:
graph = dataloader.train_graph
feat_src = node_features['item'].to(device)
graph.nodes['item'].data['h'] = feat_src

In [99]:
def similarity_matrix(X, sigma = 1.0, gamma = 2.0):
        dists = torch.cdist(X, X)
        sims = torch.exp(-dists / (sigma * dists.mean(dim = -1).mean(dim = -1).reshape(-1, 1, 1)))
        return sims
import math
import random
maxlen=20
def submodular_selection_feature(nodes):
        device = nodes.mailbox['m'].device
        mail = nodes.mailbox['m']
        batch_size, neighbor_size, feature_size = mail.shape
        cat=nodes.mailbox['c']
        user_select=[]
        for i in range(batch_size):
            select=[]
            line=cat[i].reshape(1,-1)[0].tolist()
            unique_elements = list(set(line))
            element_counts = [line.count(element) for element in unique_elements]
            element_indices = {}
            for index, element in enumerate(line):
                if element in element_indices:
                   element_indices[element].append(index)
                else:
                   element_indices[element] = [index]
            avg=math.ceil(sum(element_counts)/len(element_counts))

            sorted_indices = sorted(range(len(element_counts)), key=lambda i: element_counts[i], reverse=True)

            for i in sorted_indices:
               my_list=element_indices[unique_elements[i]]
               if int(len(my_list)/2)>avg:
                   random_elements = random.sample(my_list,int(len(my_list)/2))
               else:
                   random_elements=random.sample(my_list, k=min(avg,len(my_list)))
               select=select+random_elements
            if len(select)>=maxlen:
                select=select[0:maxlen]
            else:
                select=select+select[0:maxlen-len(select)]
            user_select.append(select)
            
        user_select=torch.tensor(user_select)

        return user_select

In [101]:

dic=[]
import math
num=[]
ma=[]
def category_aggregation(edges):
        return {'c': edges.src['category'], 'm': edges.src['h']}
def sub_reduction_item_user(nodes):
        # -1 indicate user-> node, which does not include category information
        mail = nodes.mailbox['m']
        batch_size, neighbor_size, feature_size = mail.shape
                  
    
        # if (-1 in nodes.mailbox['c']) or nodes.mailbox['m'].shape[1] <=20:
            
        #     mail=mail.sum(dim=1)
        # else:
        #     neighbors = submodular_selection_feature(nodes)     
        #     mail = mail[torch.arange(batch_size, dtype = torch.long, device = mail.device).unsqueeze(-1), neighbors]
        #     mail=mail.sum(dim=1)
        mail=mail.sum(dim=1)
        return {'h': mail}

In [102]:
with graph.local_scope():
    graph.update_all(category_aggregation, sub_reduction_item_user, etype = etype)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 7 but got size 8 for tensor number 1 in the list.